In [120]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import re
import cartopy.crs as ccrs
import wget
import subprocess

# Larger figure size
fig_size = [14, 10]
plt.rcParams['figure.figsize'] = fig_size

# https://github.com/daniestevez/jupyter_notebooks/blob/master/IONEX.ipynb
def parse_map(tecmap, exponent = -1):
    tecmap = re.split('.*END OF TEC MAP', tecmap)[0]
    return np.stack([np.fromstring(l, sep=' ') for l in re.split('.*LAT/LON1/LON2/DLON/H\\n',tecmap)[1:]])*10**exponent
    
def get_tecmaps(filename):
    with open(filename) as f:
        ionex = f.read()
        return [parse_map(t) for t in ionex.split('START OF TEC MAP')[1:]]

def get_tec(tecmap, lat, lon):
    i = round((87.5 - lat)*(tecmap.shape[0]-1)/(2*87.5))
    j = round((180 + lon)*(tecmap.shape[1]-1)/360)
    return tecmap[i,j]

def old_newformat_ionex_filename(year, day, HHMM = '0000', TYP = 'FIN', TSMP = '02H', CNT = 'GIM', centre = 'esa', zipped = True):
    # TSMP could be '02H' for every 2 hours or '01D' for every day
    # TYP could be 'FIN' for final or 'RAP' for rapid solution
    # CNT could be GIM for global ionospheric TEC maps or ROT (rate of TEC index maps)
    if zipped == True: ext = '.gz'
    else: ext = ''
    if year > 2022:
        return f'IGS0OPS{TYP}_{year:04d}{day:03d}{HHMM}_01D_{TSMP}_{CNT}.INX{ext}'
    else:
        return '{}g{:03d}0.{:02d}i{}'.format(centre, day, year % 100, '.Z' if zipped else '')

def ionex_ftp_path(year, day, centre = 'esa'):
    if year >2022:
        return 'https://cddis.nasa.gov/archive/gnss/products/ionex/{:04d}/{:03d}/{}'.format(year, day, ionex_filename(year, day))
    else:
        return 'ftp://cddis.gsfc.nasa.gov/gnss/products/ionex/{:04d}/{:03d}/{}'.format(year, day, ionex_filename(year, day, centre))

def ionex_local_path(year, day, directory = '/tmp', zipped = False):
    return directory + '/' + ionex_filename(year, day, zipped=zipped)

import os
def download_ionex(year, day, centre = 'esa', output_dir = '/tmp'):
    os.system(f'wget --auth-no-challenge -P {output_dir} "{ionex_ftp_path(year, day)}"')
    # wget.download(ionex_ftp_path(year, day, centre), output_dir)
    subprocess.call(['gzip', '-d', ionex_local_path(year, day, output_dir, zipped = True)])
    
def plot_tec_map(tecmap):
    proj = ccrs.PlateCarree()
    f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=proj))
    ax.coastlines()
    h = plt.imshow(tecmap, cmap='viridis', vmin=0, vmax=100, extent = (-180, 180, -87.5, 87.5), transform=proj)
    plt.title('VTEC map')
    divider = make_axes_locatable(ax)
    ax_cb = divider.new_horizontal(size='5%', pad=0.1, axes_class=plt.Axes)
    f.add_axes(ax_cb)
    cb = plt.colorbar(h, cax=ax_cb)
    plt.rc('text', usetex=True)
    cb.set_label('TECU ($10^{16} \\mathrm{el}/\\mathrm{m}^2$)')

In [122]:
ionex_ftp_path(2024, 1, 'esa')

TypeError: ionex_filename() missing 1 required positional argument: 'centre'

In [133]:
def ionex_filename(year, day, centre, zipped = True):
    return '{}g{:03d}0.{:02d}i{}'.format(centre, day, year % 100, '.Z' if zipped else '')

def ionex_ftp_path(year, day, centre):
    # print('ftp://cddis.gsfc.nasa.gov/gnss/products/ionex/{:04d}/{:03d}/{}'.format(year, day, ionex_filename(year, day, centre)))
    return 'https://cddis.nasa.gov/archive/gnss/products/ionex/{:04d}/{:03d}/{}'.format(year, day, ionex_filename(year, day, centre))
    # return 'ftp://cddis.gsfc.nasa.gov/gnss/products/ionex/{:04d}/{:03d}/{}'.format(year, day, ionex_filename(year, day, centre))

def ionex_local_path(year, day, centre = 'esa', directory = '/tmp', zipped = False):
    return directory + '/' + ionex_filename(year, day, centre, zipped)
    
def download_ionex(year, day, centre = 'esa', output_dir = '/tmp'):
    print(ionex_ftp_path(year, day, centre))
    os.system(f'wget --auth-no-challenge -P {output_dir} "{ionex_ftp_path(year, day, centre)}"')

    # wget.download(ionex_ftp_path(year, day, centre), output_dir)
    subprocess.call(['gzip', '-d', ionex_local_path(year, day, centre, output_dir, zipped = True)])

In [36]:
# YYYY/DDD/AAAgDDD#.YYi.Z

In [47]:
ionex_ftp_path(2022, 1, 'esa')

'ftp://cddis.gsfc.nasa.gov/gnss/products/ionex/2022/001/esag0010.22i.Z'

In [134]:
download_ionex(2022, 330, output_dir='/Users/rdcrlzh1/Documents/SWE_error_analysis/local/ionosphere')

https://cddis.nasa.gov/archive/gnss/products/ionex/2022/330/esag3300.22i.Z


--2025-06-03 11:04:33--  https://cddis.nasa.gov/archive/gnss/products/ionex/2022/330/esag3300.22i.Z
Resolving cddis.nasa.gov (cddis.nasa.gov)... 198.118.199.52, 2001:4d0:241a:442::52
Connecting to cddis.nasa.gov (cddis.nasa.gov)|198.118.199.52|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://urs.earthdata.nasa.gov/oauth/authorize?client_id=gDQnv1IO0j9O2xXdwS8KMQ&response_type=code&redirect_uri=https%3A%2F%2Fcddis.nasa.gov%2Fproxyauth&state=aHR0cDovL2NkZGlzLm5hc2EuZ292L2FyY2hpdmUvZ25zcy9wcm9kdWN0cy9pb25leC8yMDIyLzMzMC9lc2FnMzMwMC4yMmkuWg [following]
--2025-06-03 11:04:34--  https://urs.earthdata.nasa.gov/oauth/authorize?client_id=gDQnv1IO0j9O2xXdwS8KMQ&response_type=code&redirect_uri=https%3A%2F%2Fcddis.nasa.gov%2Fproxyauth&state=aHR0cDovL2NkZGlzLm5hc2EuZ292L2FyY2hpdmUvZ25zcy9wcm9kdWN0cy9pb25leC8yMDIyLzMzMC9lc2FnMzMwMC4yMmkuWg
Resolving urs.earthdata.nasa.gov (urs.earthdata.nasa.gov)... 198.118.243.33, 2001:4d0:241a:4081::89
Connecting to urs.earthd

In [107]:
year = 2023
day = 1
zipped = True
ext = '.gz' if zipped == True else ''
HHMM = '0000'
TYP = 'FIN'
TSMP = '02H'
CNT = 'GIM'
centre = 'esa'

'{}g{:03d}0.{:02d}i{}'.format(centre, day, year % 100, '.Z' if zipped else '')
    

'esag0010.23i.Z'

In [109]:
f'{centre}g{day:03d}0.{year%100:02d}i{ext}'

'esag0010.23i.gz'

In [155]:
from pathlib import Path

def get_ionex_filename(year, day, old_format, HHMM = '0000', TYP = 'FIN', TSMP = '02H', CNT = 'GIM', cente = 'esa', zipped = True):
    if old_format:
        ext = '.Z' if zipped else ''
        return f'{centre}g{day:03d}0.{year%100:02d}i{ext}'
    else:
        ext = '.gz' if zipped else ''
        return f'IGS0OPS{TYP}_{year:04d}{day:03d}{HHMM}_01D_{TSMP}_{CNT}.INX{ext}'

from urllib.parse import urljoin
import shutil
def download_ionex(year, day, output_dir, center = 'esa'):
    if isinstance(output_dir, str): output_dir = Path(output_dir)
    old_format = True if year <= 2022 and day <= 330 else False
    
    main_https = 'https://cddis.nasa.gov/archive/gnss/products/ionex/'
    filename = get_ionex_filename(year, day, old_format)
    https_fp = urljoin(main_https, f'{year:04d}/{day:03d}/{filename}')
    print(https_fp)
    os.system(f'wget --auth-no-challenge -P {output_dir} "{https_fp}"')
    subprocess.call(['gzip', '-d', output_dir.joinpath(filename)])

    if old_format:
        old_format_fp = output_dir.joinpath(get_ionex_filename(year, day, old_format, zipped = False))
        new_format_fp = output_dir.joinpath(get_ionex_filename(year, day, old_format = False, zipped = False))
        shutil.move(old_format_fp, new_format_fp)

In [ ]:
# https://cddis.nasa.gov/archive/gnss/products/ionex/2016/001/esag0010.16i.Z
# https://cddis.nasa.gov/archive/gnss/products/ionex/2016/000/esag0000.16i.Z

In [156]:
download_ionex(2016, 1, '/Users/rdcrlzh1/Documents/SWE_error_analysis/local/ionosphere')

https://cddis.nasa.gov/archive/gnss/products/ionex/2016/001/esag0010.16i.Z


--2025-06-03 11:17:21--  https://cddis.nasa.gov/archive/gnss/products/ionex/2016/001/esag0010.16i.Z
Resolving cddis.nasa.gov (cddis.nasa.gov)... 198.118.199.52, 2001:4d0:241a:442::52
Connecting to cddis.nasa.gov (cddis.nasa.gov)|198.118.199.52|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://urs.earthdata.nasa.gov/oauth/authorize?client_id=gDQnv1IO0j9O2xXdwS8KMQ&response_type=code&redirect_uri=https%3A%2F%2Fcddis.nasa.gov%2Fproxyauth&state=aHR0cDovL2NkZGlzLm5hc2EuZ292L2FyY2hpdmUvZ25zcy9wcm9kdWN0cy9pb25leC8yMDE2LzAwMS9lc2FnMDAxMC4xNmkuWg [following]
--2025-06-03 11:17:21--  https://urs.earthdata.nasa.gov/oauth/authorize?client_id=gDQnv1IO0j9O2xXdwS8KMQ&response_type=code&redirect_uri=https%3A%2F%2Fcddis.nasa.gov%2Fproxyauth&state=aHR0cDovL2NkZGlzLm5hc2EuZ292L2FyY2hpdmUvZ25zcy9wcm9kdWN0cy9pb25leC8yMDE2LzAwMS9lc2FnMDAxMC4xNmkuWg
Resolving urs.earthdata.nasa.gov (urs.earthdata.nasa.gov)... 198.118.243.33, 2001:4d0:241a:4081::89
Connecting to urs.earthd